In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
import os
from Bio.Blast.Applications import NcbiblastnCommandline
import re
import inspect
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sklearn as sklearn
import seaborn as sns; sns.set()
import scipy
from sklearn.linear_model import Lasso
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib
import json
from scipy import stats
import math

In [ ]:
#Used for probe creation
class BlastResult:
    def __init__(self, qseqid, sseqid, pident, length, mismatch, gapopen, qstart, qend, 
                 sstart, send, evalue, bitscore):
        self.qseqid = qseqid
        self.sseqid = sseqid
        self.pident = pident
        self.length = int(length)
        self.mismatch = int(mismatch)
        self.gapopen = int(gapopen)
        self.qstart = int(qstart)
        self.qend = int(qend)
        self.sstart = int(sstart)
        self.send = int(send)
        self.evalue = float(evalue)
        self.bitscore = float(bitscore)

In [ ]:
def save_figure(figure_to_save, figure_save_path, sample_name, figure_name):
    figure_to_save.savefig(os.path.join(figure_save_path, (sample_name + '_' + figure_name)), 
                                     format = 'eps', dpi = 500)
#Functions for generating gene list
def L1_regression(sample_data, single_cell_data, pseudotime_correlated_gene_list, desired_num_genes,
                  genes_to_avoid):
    X = single_cell_data[pseudotime_correlated_gene_list]
    X = X.drop(genes_to_avoid, axis = 1)
    Y = sample_data['Pseudotime']
    X_train,X_test,y_train,y_test=train_test_split(X,Y, test_size=0.3, random_state=31)
    coeff_used = 0
    i = 500.0
    minimum = 0.0
    while coeff_used != desired_num_genes:
        lasso = Lasso(alpha = i, max_iter=10e5)
        lasso.fit(X_train,y_train)
        coeff_used = np.sum(lasso.coef_!=0)
        if coeff_used > desired_num_genes:
            minimum = i
            i = (i+maximum)/2
        if coeff_used < desired_num_genes:
            maximum = i
            i = (i+minimum)/2
    
    train_score =lasso.score(X_train,y_train)
    test_score=lasso.score(X_test,y_test)
    coefficients = lasso.sparse_coef_
    return train_score, test_score, coefficients

def create_gene_dictionaries(pseudotime_correlated_gene_list, L1_gene_list):
    negative_gene_to_coefficient_dict = {}
    positive_gene_to_coefficient_dict = {}
    for i in range(0,L1_gene_list.shape[1]):
        if L1_gene_list[0,i] < 0:
            negative_gene_to_coefficient_dict[str(pseudotime_correlated_gene_list[i])] = L1_gene_list[0,i]
        if L1_gene_list[0,i] > 0:
            positive_gene_to_coefficient_dict[str(pseudotime_correlated_gene_list[i])] = L1_gene_list[0,i]
        else:
            pass
    return negative_gene_to_coefficient_dict, positive_gene_to_coefficient_dict
    
def num_coefficients_vs_rsqaured_graph(sample_data, single_cell_data, min_num_genes_to_test, 
                                       max_num_genes_to_test, pseudotime_correlated_gene_list, genes_to_avoid):
    scores = list()
    for j in tqdm(range(min_num_genes_to_test,max_num_genes_to_test+1)):
        train_score, test_score, coefficients = L1_regression(sample_data = sample_data, 
                                 single_cell_data = single_cell_data, 
                                 pseudotime_correlated_gene_list = pseudotime_correlated_gene_list, 
                                 desired_num_genes = j, genes_to_avoid = genes_to_avoid)
        scores.append(test_score)
    fig = plt.figure(figsize=(8, 6))
    plt.scatter(range(min_num_genes_to_test,max_num_genes_to_test+1), scores)
    plt.xlabel('Number of Genes')
    plt.ylabel('$R^{2}$')
    plt.show()
    return fig
    
def save_dictionaries(data_path, sample_name, positive_gene_to_coefficient_dict
                      , negative_gene_to_coefficient_dict):
    negative_gene_to_coefficient_dict = {'negative_gene_to_coefficient_dict':negative_gene_to_coefficient_dict}
    neg_dict_path = os.path.join(file_path, sample_name, 'negative_gene_coeff_dict.txt')
    with open(neg_dict_path, 'w') as file:
        file.write(json.dumps(negative_gene_to_coefficient_dict))
    positive_gene_to_coefficient_dict = {'positive_gene_to_coefficient_dict' + positive_gene_to_coefficient_dict}
    pos_dict_path = os.path.join(file_path, sample_name, 'positive_gene_coeff_dict.txt')
    with open(pos_dict_path, 'w') as file:
        file.write(json.dumps(positive_gene_to_coefficient_dict))
        
def generate_predicted_pseudotime_values(sample_data, single_cell_data, positive_gene_to_coefficient_dict, 
                                  negative_gene_to_coefficient_dict):
    sample_data['negative_coefficients_sum'] = 0
    sample_data['positive_coefficients_sum'] = 0
    sample_data['predicted_pseudotime'] = 0
    for i in negative_gene_to_coefficient_dict:
        sample_data['negative_coefficients_sum'] = (sample_data['negative_coefficients_sum'] + 
                                                    (single_cell_data[i] * 
                                                    negative_gene_to_coefficient_dict.get(i)))
    for j in positive_gene_to_coefficient_dict:
        sample_data['positive_coefficients_sum'] = (sample_data['positive_coefficients_sum'] + 
                                                    (single_cell_data[j] * 
                                                    positive_gene_to_coefficient_dict.get(j)))
    sample_data['predicted_pseudotime'] = (sample_data['negative_coefficients_sum'] 
                                               + sample_data['positive_coefficients_sum'])
    return sample_data

def generate_predicted_pseudotime_values_from_loaded_gene_list(sample_data, single_cell_data, 
                                                               positive_gene_to_coefficient_dict, 
                                                                negative_gene_to_coefficient_dict):
    sample_data['negative_coefficients_sum'] = 0
    sample_data['positive_coefficients_sum'] = 0
    sample_data['predicted_pseudotime'] = 0
    for i in negative_gene_to_coefficient_dict['negative_gene_to_coefficient_dict']:
        sample_data['negative_coefficients_sum'] = (sample_data['negative_coefficients_sum'] + 
                                                    (single_cell_data[i] * 
                                negative_gene_to_coefficient_dict['negative_gene_to_coefficient_dict'].get(i)))
    for j in positive_gene_to_coefficient_dict['positive_gene_to_coefficient_dict']:
        sample_data['positive_coefficients_sum'] = (sample_data['positive_coefficients_sum'] + 
                                                    (single_cell_data[j] * 
                                positive_gene_to_coefficient_dict['positive_gene_to_coefficient_dict'].get(j)))
    sample_data['predicted_pseudotime'] = (sample_data['negative_coefficients_sum'] 
                                               + sample_data['positive_coefficients_sum'])
    return sample_data

def plot_predicted_pseudotime(sample_data_with_predicted_values):
    fig = plt.figure(figsize = (8,6))
    plt.scatter(sample_data_with_predicted_values['negative_coefficients_sum']*-1,
                sample_data_with_predicted_values['positive_coefficients_sum'],
                c = sample_data_with_predicted_values['Pseudotime'])
    plt.title("Predicted Pseudotime Values Colored by Original Pseudotime Values")
    plt.xlabel("Negative Coefficients Sum of Absolute Values")
    plt.ylabel("Positive Coefficients Sum")
    plt.colorbar()
    return fig

def get_probe_numbers(negative_gene_to_coefficient_dict, positive_gene_to_coefficient_dict, 
                      max_negative_probes, max_positive_probes):
    neg_max = 0
    pos_max = 0
    for i in negative_gene_to_coefficient_dict:
        if negative_gene_to_coefficient_dict.get(i)*-1 > neg_max:
                neg_max = negative_gene_to_coefficient_dict.get(i)*-1
    for j in positive_gene_to_coefficient_dict:
        if positive_gene_to_coefficient_dict.get(j) > pos_max:
            pos_max = positive_gene_to_coefficient_dict.get(j)
    negative_gene_to_probe_num_dict = {}
    positive_gene_to_probe_num_dict = {}
    for i in negative_gene_to_coefficient_dict:
        negative_gene_to_probe_num_dict[i] = (max_negative_probes/neg_max*
                                              negative_gene_to_coefficient_dict.get(i)*-1)
    for j in positive_gene_to_coefficient_dict:
        positive_gene_to_probe_num_dict[j] = max_positive_probes/pos_max*positive_gene_to_coefficient_dict.get(j)
    return negative_gene_to_probe_num_dict, positive_gene_to_probe_num_dict


def reverse_complement(seq):
    seq = Seq.upper(seq)
    bases = (COMPLEMENT_MAP[base] for base in reversed(seq))
    bases = ''.join(bases)
    return bases

def has_hairpin(temp_seq, neck_length, loop_length):
    temp_seq = Seq.upper(temp_seq)
    start_point = 0
    while start_point <= len(temp_seq)-neck_length-loop_length:
        to_check = temp_seq[start_point+neck_length+loop_length:]
        temp_sub_rev_comp = reverse_complement(temp_seq[start_point:start_point+neck_length])
        if temp_sub_rev_comp in to_check:
            return True
        else:
            start_point += 1
    return False

def has_dimer(temp_seq, max_overlap):
    temp_seq = Seq.upper(temp_seq)
    start_point = 0
    rev_comp = reverse_complement(temp_seq)
    while start_point <= len(temp_seq)-max_overlap*2:
        to_check = temp_seq[start_point:]
        temp_sub_rev_comp = reverse_complement(temp_seq[start_point:start_point+max_overlap])
        if temp_sub_rev_comp in to_check:
            return True
        else:
            start_point += 1
    return False

def has_improper_GC(temp_seq, min_GC, max_GC):
    temp_seq = Seq.upper(temp_seq)
    GC_total = 0
    for i in temp_seq:
        if (i is 'G' or i is'C'):
            GC_total += 1
    GC_percent = float(GC_total)/len(temp_seq)*100
    if (GC_percent < min_GC or GC_percent > max_GC):
        return True
    return False

# Beginning of workflow for generating gene list
### Provide values for the following variables:
    data_path: Path to directory in which all files that will be used for analysis are stored
    
    to_save_path: Directory to which you would like newly generated data to be saved
    
    single_cell_dge_file: Absolute expression count matrix where columns are genes and rows are cells
    
    pseudotime_correlated_genes_file: Text file containing genes with expression correlated with pseudotime
    
    sample_data_file: File containing the cells and their generated pseudotime values
    
    singel_cell_data_name: Name of sample/data to be used when saving newly generated data

In [ ]:
data_path = '/Users/Data/Path'
single_cell_dge_file = 'Your_scRNA_seq_dataset_gene_matrix.csv'
pseudotime_correlated_genes_file = 'Your_1000_pseudotime_correlated_gene_list.txt'
sample_data_file = 'Your_pseudotime_value_file.txt'
figure_save_path = '/Users/output/data/path'
single_cell_data_name = 'your_experiment_name'

In [ ]:
single_cell_expression_matrix = pd.read_csv(os.path.join(data_path,single_cell_dge_file), 
                                                   header = 0, index_col = 0)

In [ ]:
pseudotime_correlated_genes = pd.read_csv(os.path.join(data_path,pseudotime_correlated_genes_file), sep = '\t')
pseudotime_correlated_genes_list = list(pseudotime_correlated_genes.index)

In [ ]:
sample_data = pd.read_csv(os.path.join(data_path, sample_data_file), delimiter = '\t')

### Generate graph of number of genes used in regression vs predictive ability (represented by $R^{2}$)
Generate graph of number of genes used in linear regression vs $R^{2}$ to help decide how many genes should be used

    Warning: min_num_genes_to_test values that are too low may result in errors

In [ ]:
genes_to_avoid = list()
genes_to_avoid.append('genes_you_want_to_exclude')

genes_vs_rsqaured_graph = num_coefficients_vs_rsqaured_graph(sample_data = sample_data, 
                                           single_cell_data = single_cell_expression_matrix,
                                           min_num_genes_to_test = 4, max_num_genes_to_test = 20,
                                           pseudotime_correlated_gene_list = pseudotime_correlated_genes_list,
                                            genes_to_avoid = genes_to_avoid)

### Run L1 regularized linear regression to generate gene list of desired length

In [ ]:
train_score, test_score, coefficients = L1_regression(sample_data = sample_data,
                                             single_cell_data = single_cell_expression_matrix,
                                             pseudotime_correlated_gene_list = pseudotime_correlated_genes_list,
                                             genes_to_avoid = genes_to_avoid, desired_num_genes = 14)


print('training set score: ' + str(train_score) + '\n'+ 'testing set score: ' + str(test_score))

### Create dictionaries for the generated coefficients
Creates two dictionaries with genes as keys and their corresponding coefficients as generated by L1. One dictionary cointains all the genes with negative coefficients and the other with all the positive

In [ ]:
negative_gene_to_coefficient_dict, positive_gene_to_coefficient_dict = create_gene_dictionaries(
                                            pseudotime_correlated_gene_list = pseudotime_correlated_genes_list, 
                                            L1_gene_list = coefficients)
print(negative_gene_to_coefficient_dict)
print(positive_gene_to_coefficient_dict)

### Visualize predicted pseudotime against actual pseudotime

In [ ]:
sample_data_with_predictions = generate_predicted_pseudotime_values(sample_data = sample_data,
                                    single_cell_data = single_cell_expression_matrix,
                                    positive_gene_to_coefficient_dict = positive_gene_to_coefficient_dict,
                                    negative_gene_to_coefficient_dict = negative_gene_to_coefficient_dict)

predicted_pseudotime_vs_actual_figure = plot_predicted_pseudotime(sample_data_with_predicted_values
                                                      = sample_data_with_predictions)

### Save dictionary files
Creates two seperate dictionary text files, one for the negative coefficients and one for the positive

In [ ]:
### Optional: Determine number of HCR probes necessary for each gene
Sets the highest coefficient in each dictionary to desired max number of probes per gene and adjusts remaining 

max_negative/positive_probes = maximum number of probes that should be designed to bind to one gene from the negative/positives coefficient dictionary

In [ ]:
max_negative_probes = 20
max_positive_probes = 20
negative_gene_to_probe_num_dict, positive_gene_to_probe_num_dict = get_probe_numbers( 
                                        negative_gene_to_coefficient_dict = negative_gene_to_coefficient_dict, 
                                        positive_gene_to_coefficient_dict = positive_gene_to_coefficient_dict, 
                                        max_negative_probes = max_negative_probes, 
                                        max_positive_probes = max_positive_probes)
print('negative gene to probe dictionary:')
print(negative_gene_to_probe_num_dict)
print('positive gene to probe dictionary:')
print(positive_gene_to_probe_num_dict)

### Optional: Load in and test other gene lists on other data sets

In [ ]:
# path_to_directory_with_dictionaries = '/Users/Daniel/Desktop/Summer_Research/SpermData/MoreData/GSE104556(GOOD)/''
# negative_gene_coefficient_dict_file = 'Positive_coefficients_Other2.txt'
# positive_gene_coefficient_dict_file = 'Negative_coefficients_Other2.txt'
# with open(os.path.join(path_to_directory_with_dictionaries, negative_gene_coefficient_dict_file)) as json_file:  
#     outside_negative_gene_to_coefficient_dict = json.load(json_file)
# with open(os.path.join(path_to_directory_with_dictionaries, positive_gene_coefficient_dict_file)) as json_file:  
#     outside_positive_gene_to_coefficient_dict = json.load(json_file)

# Creating probes

To generate probes first make sure you have installed ncbi tools for performing local blast

Create an excel file called 'GeneList.xlsx' with the first column the common gene name (i.e. Prm1) and the NCBI accession number (i.e. NM_013637.5)

Next, download all of the sequences in the genbank file format to a single file called 'sequence_marker.gb' 

    probelength = length of each probe
    spacerlength = minimum distance between probes
    GC_cutoff =[minimum GC content tolerated in aprobe, maximum GC content tolerated in a probe]

In [ ]:
probelength = 52
spacerlength = 3
GC_cutoff = [40, 66]

In [ ]:
COMPLEMENT_MAP = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', '-':'-', 'N':'N'}

In [ ]:
gene_frame = pd.read_excel('GeneList.xlsx')

In [ ]:
gb_file = 'sequence_marker.gb'
gene_to_sequence_dict = {}

In [ ]:
for gb_record in SeqIO.parse(open(gb_file,"r"), "genbank"):
    gb_record.features = [f for f in gb_record.features if f.type == "CDS"]
    start = gb_record.features[0].location.start.position
    gene_to_sequence_dict[gb_record.name] = gb_record.seq[start::]

Running the following code generates a file for each gene that contains all the possible probe sequences. Files are saved as commonname_accession_id.fasta

(i.e. Prm1_NM_013636.fasta)

In [ ]:
for index, accession in enumerate(gene_frame.iloc[:,1]):
    probe_num = 1
    generic_name = (gene_frame.iloc[index,0])
    if accession:
        accession_trim = accession[0:accession.find('.')]
        print accession
    S = gene_to_sequence_dict[accession_trim]
    print len(S)
    seq_list = list()
    index_list = list()
    tail = probelength + spacerlength
    while tail < len(S):
        temp_seq = S[(tail-probelength-spacerlength):tail-spacerlength]
        to_save = SeqRecord(temp_seq, str(index))
        SeqIO.write(to_save, "temp_seq.fasta", "fasta")
        hits = False
        blast_hits = False
        #print temp_seq
        if Seq("CCCCC") in temp_seq:
            hits = True
        elif Seq("GGGGG") in temp_seq:
            hits = True
        elif Seq("AAAAA") in temp_seq:
            hits = True
        elif Seq("TTTTT") in temp_seq:
            hits = True
        #BSRDI restriction site exclusion
        #if Seq("GCAATG") in temp_seq:
            #hits = True
        #BSAI restriction site exclusion
        elif Seq("GGTCTC") in temp_seq:
            hits = True
        elif Seq("GAGACC") in temp_seq:
            hits = True
        elif has_hairpin(temp_seq,7,6):
            hits = True
        elif has_dimer(temp_seq, 10):
            hits = True
        elif has_improper_GC(temp_seq, GC_cutoff[0], GC_cutoff[1]):
            hits = True
        if hits == False:
            output = NcbiblastnCommandline(query="temp_seq.fasta", db="mouserefseq.fna", outfmt = 6, 
                                    strand = "plus", dust = "no", penalty = -3, word_size = 7, gapopen = 5, 
                                    gapextend = 2, reward = 1, max_target_seqs = 50, perc_identity = 25)
            stdout, stderr = output()
            blast_output = stdout.splitlines()
            blast_result_list = list()
            for i in xrange(len(blast_output)):
                blast_output[i] = blast_output[i].split('\t')
                blast_result_list.append(BlastResult(*(j for j in blast_output[i])))
            for blast_result in blast_result_list:
                if accession_trim not in blast_result.sseqid: #ignores same gene
                    if 'X' not in blast_result.sseqid: #ignores predicted
                        if (blast_result.qstart < 12 and blast_result.qend > 20): #check if well centered
                            blast_hits = True
                            print temp_seq
                            print reverse_complement(temp_seq)
                            print blast_result.qstart
                            print blast_result.qend
        os.remove('temp_seq.fasta')
        if blast_hits == True:
            print 'hi'
            tail += 8
        if hits == True:
            tail += 5
        if hits == False:
            probe_num += 1
            #print(probe_num)
            #print(tail-probelength-spacerlength+1)
            tail = tail + probelength + spacerlength
            index_list.append(tail-probelength-spacerlength)
            seq_list.append(reverse_complement(temp_seq))
        if tail >= len(S):
            final_probes = (SeqRecord(Seq(probe_seq), str(index+1), 
                        description = ("probe number " + str(index+ 1) + '     index ' + str(index_list[index]))) 
                            for index, probe_seq in enumerate(seq_list))
            SeqIO.write(final_probes, generic_name +'_'+accession_trim+ '.fasta', 'fasta')